# Agenda - Sept 11th 
    - Lexie


- Data cleaning
    - EDA: exploratory data analysis
    - Build up-to-date medical dictionary based on our current answers by doctors
    

- Clustering - kmeans/LDA in topic modeling 
    - get similarity list of a group of answers
    

- Feature extraction 
    - drug names, treatment, disease... 
    - question level and user level
    
    
- Question analysis 
    - based on question setting, classify each question into different groups
    

- Next step: relation extraction
    - get to know more in text(answers)

In [5]:
import pandas as pd
import numpy as np
import pymysql
import re
pd.set_option('display.max_colwidth', -1)
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer ##文本向量化
from sklearn.decomposition import LatentDirichletAllocation # LDA
import psycopg2 #postgresql
import pyLDAvis
import pyLDAvis.sklearn
import pymetamap
from pymetamap import MetaMap
import string
import psycopg2

In [6]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
config.sections()

sos_db_section = config['SOS_DB']
host = sos_db_section['host']
user = sos_db_section['user']
password = sos_db_section['password']
db=sos_db_section['db']
port = int(sos_db_section.get('port', 3306))
db = pymysql.connect(host=host, user=user, password=password, db=db, port=port)

In [7]:
import_db_section = config['docdx_db_production']
host = import_db_section['host']
user = import_db_section['user']
password = import_db_section['password']
db_dx=import_db_section['db']
port = int(sos_db_section.get('port',5432))

conn_str = "host={} dbname={} user={} password={}".format(host, db_dx, user, password)
db_docdx = psycopg2.connect(conn_str)

## 1. Load Data 
- Docdx

In [206]:
#name: topic specialty
#spe: doctor's specialty 
query_docdx = """select CC.npi, BB.first_name, BB.last_name, BB.email, AA.created_at,topic.id, topic.title,topic.description, AA.text, DD.name, GG.spe from comment AA 
inner join public."user" BB 
on AA.created_by = BB.id 
inner join user_profile CC
on CC.user_id = AA.created_by 
inner join topic 
on topic.id = AA.topic_id
inner join 
(select specialty.name, topic_specialty.topic_id from topic_specialty
inner join specialty 
on topic_specialty.specialty_id = specialty.id) DD
on DD.topic_id = AA.topic_id
inner join (select CC.user_id user_id_,EE.name spe from public."user_profile" CC
inner join specialty EE
on EE.id = CC.specialty_id
inner join subspecialty FF
on FF.id = CC.subspecialty_id) GG
on GG.user_id_ = AA.created_by
where AA.reply_to_comment_id is null and AA.deleted_at is null 
order by AA.created_at asc"""
df_docdx = pd.read_sql(query_docdx, db_docdx)

In [207]:
len(df_docdx)

10313

### 1.1 Check valid data 
- test_topics
- topic with multiple specialties

In [208]:
print('Specialty names')
df_docdx.name.unique()

Specialty names


array(['Dermatology', 'Gastroenterology', 'OTO / ENT', 'Pulmonology',
       'Neurology', 'Oncology / Hematology', 'Rheumatology',
       'UNSPECIFIED / OTHER', 'Anesthesia / Pain', 'Internal Medicine',
       'Endocrinology / Diabetes', 'Family Practice', 'Cardiology',
       'Ophthalmology / Optometrists', 'Allergy / Immunology',
       'Nephrology', 'Radiology', 'Psychiatry', 'Urology',
       'Nurse Practitioners', 'Surgeons', 'Pediatrics'], dtype=object)

In [209]:
#prg_id in different types 
query_multiple_topic = """select topic_id, count(topic_id) from topic_specialty
group by topic_id
having count(topic_id) > 1"""
df_topic_multiple_count = pd.read_sql(query_multiple_topic, db_docdx)

In [11]:
df_topic_multiple_count

,topic_id,count
0,16,3
1,148,2
2,189,9
3,3,4
4,67,2
5,107,2
6,147,2
7,145,21


In [12]:
query_multiple_topic_check = """select topic.title,topic.description, specialty.name, topic_specialty.topic_id from topic_specialty
inner join specialty 
on topic_specialty.specialty_id = specialty.id
inner join topic
on topic.id = topic_specialty.topic_id
where topic_id in 
(select topic_id from topic_specialty
group by topic_id
having count(topic_id) > 1) 
order by topic_id """
df_topic_multiple = pd.read_sql(query_multiple_topic_check, db_docdx)

In [14]:
df_topic_multiple.head()

,title,description,name,topic_id
0,Test Question edit,This is just a test question so I can see if moderator can create a new question on production. Will delete it right away! edit,Anesthesia / Pain,3
1,Test Question edit,This is just a test question so I can see if moderator can create a new question on production. Will delete it right away! edit,Cardiology,3
2,Test Question edit,This is just a test question so I can see if moderator can create a new question on production. Will delete it right away! edit,Dermatology,3
3,Test Question edit,This is just a test question so I can see if moderator can create a new question on production. Will delete it right away! edit,Allergy / Immunology,3
4,Testteteteteqweqe,asdasddasdasdasdsasdadasdaedit,Anesthesia / Pain,16


In [210]:
#remove test topics
test_topic = [3,16,67,23,24,25,26,27,28,29]
df_docdx = df_docdx[~df_docdx['id'].isin(test_topic)]

In [211]:
#test topics removal 
len(df_docdx)

10306

- Pick one specialty in the multi-topic cases

In [212]:
#mapped to dataset 
def one_name(id):
    if id == 189:
        return 'Oncology / Hematology'
    elif id == 145:
        return 'Internal Medicine'
    elif id == 107:
        return 'Family Practice'
    elif id == 180:
        return 'Pulmonology'
    elif id == 147:
        return 'Gastroenterology'
    elif id == 148:
        return 'Oncology / Hematology' 
    else:
        return str(df_docdx[df_docdx['id'] == id].name.unique())[2:-2]

In [213]:
df_docdx['name'] = df_docdx['id'].apply(lambda x: one_name(x))

In [214]:
len(df_docdx)

10306

In [215]:
#remove dupe in the df_docdx 
#because we we get it from the database, every answer will have multiple specialties mapped 
df_docdx = df_docdx.drop_duplicates(subset = ['id', 'text','npi'])

In [216]:
#remove pattern
pattern_new = "\n"
df_docdx['text'] = df_docdx.text.apply(lambda x: re.sub(pattern_new, " ", x))

#remove punctuation 

def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

df_docdx['text_np'] = df_docdx['text'].apply(lambda x: remove_punctuation(x))

#change to lower case 
df_docdx['text_np'] = df_docdx['text_np'].str.lower()
#df_docdx['text'] = df_docdx['text'].str.lower()

In [221]:
#only the info we want
#not in a user level 
df_docdx_clean = df_docdx[['id','title','description','text_np','name','spe']]

In [222]:
df_docdx_clean.head(5)

,id,title,description,text_np,name,spe
0,44,Patient Case: would you confirm Blastic Plasmacytoid Dendritic Cell Neoplasm?,"A 65-year-old male patient presents with a 1-month history of multiple asymptomatic bruise-like lesions on his trunk and face. Additionally, he developed ecchymotic lesions on the trunk easily from scratching. He also suffered from several episodes of bleeding in the right nasal cavity for 1-2 months. He denied having any underlying systemic disease or malignancy. On physical examination, the following were noted: multiple 2-3 cm in diameter bluish to violaceous infiltrated patches or plaques scattered on his trunk and a few irregularly shaped violaceous plaques on his cheeks. Based on these symptoms, how would you proceed to either confirm or rule out a diagnosis of Blastic Plasmacytoid Dendritic Cell Neoplasm (BPDCN)? If patient is diagnosed with BPDCN, what would be the typical treatment approach for him?",refer to oncologist,Dermatology,Dermatology
1,44,Patient Case: would you confirm Blastic Plasmacytoid Dendritic Cell Neoplasm?,"A 65-year-old male patient presents with a 1-month history of multiple asymptomatic bruise-like lesions on his trunk and face. Additionally, he developed ecchymotic lesions on the trunk easily from scratching. He also suffered from several episodes of bleeding in the right nasal cavity for 1-2 months. He denied having any underlying systemic disease or malignancy. On physical examination, the following were noted: multiple 2-3 cm in diameter bluish to violaceous infiltrated patches or plaques scattered on his trunk and a few irregularly shaped violaceous plaques on his cheeks. Based on these symptoms, how would you proceed to either confirm or rule out a diagnosis of Blastic Plasmacytoid Dendritic Cell Neoplasm (BPDCN)? If patient is diagnosed with BPDCN, what would be the typical treatment approach for him?",patient needs a skin biopsy not sure how to treat though,Dermatology,Dermatology
2,44,Patient Case: would you confirm Blastic Plasmacytoid Dendritic Cell Neoplasm?,"A 65-year-old male patient presents with a 1-month history of multiple asymptomatic bruise-like lesions on his trunk and face. Additionally, he developed ecchymotic lesions on the trunk easily from scratching. He also suffered from several episodes of bleeding in the right nasal cavity for 1-2 months. He denied having any underlying systemic disease or malignancy. On physical examination, the following were noted: multiple 2-3 cm in diameter bluish to violaceous infiltrated patches or plaques scattered on his trunk and a few irregularly shaped violaceous plaques on his cheeks. Based on these symptoms, how would you proceed to either confirm or rule out a diagnosis of Blastic Plasmacytoid Dendritic Cell Neoplasm (BPDCN)? If patient is diagnosed with BPDCN, what would be the typical treatment approach for him?",i would biopsy a lesion chemotherapy can be used for treatment prognosis is poor,Dermatology,Dermatology
3,44,Patient Case: would you confirm Blastic Plasmacytoid Dendritic Cell Neoplasm?,"A 65-year-old male patient presents with a 1-month history of multiple asymptomatic bruise-like lesions on his trunk and face. Additionally, he developed ecchymotic lesions on the trunk easily from scratching. He also suffered from several episodes of bleeding in the right nasal cavity for 1-2 months. He denied having any underlying systemic disease or malignancy. On physical examination, the following were noted: multiple 2-3 cm in diameter bluish to violaceous infiltrated patches or plaques scattered on his trunk and a few irregularly shaped violaceous plaques on his cheeks. Based on these symptoms, how would you proceed to either confirm or rule out a diagnosis of Blastic Plasmacytoid Dendritic Cell Neoplasm (BPDCN)? If patient is diagnosed with BPDCN, what would be the typical treatment approach for him?",a skin biopsy labs and oncology consult asap,Dermatology,Dermatology
4,44,Patient Case: would you confirm Blast

## 2. K-means - Docdx

https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68

- Clustering 
    - Clustering is a method of unsupervised learning and is a common technique for statistical data analysis used in many fields.
    - In Data Science, we can use clustering analysis to gain some valuable insights from our data by seeing what groups the data points fall into when we apply a clustering algorithm. 
    
- Word2vec + kmeans


In [108]:
from sklearn.cluster import KMeans

In [78]:
#create a dictionary on base on question/answers with their mapped specialty 
#{id: [answer_1, answer_2....] }
answers_dict = dict()
for index, row in df_docdx_clean.iterrows():
    qid = row['id']
    answer = row['text_np']
    #print(row['text_np'])
    if qid in answers_dict :
        answers_dict[qid].append(answer)
    else : 
        answers_dict[qid] = [answer]

In [76]:
#create a dictionary: 
#{specialty: question_id}
topic_spec_dict = {}
for index, row in df_docdx_clean.drop_duplicates(subset = ['id', 'title','name']).iterrows():
    spec = row['name']
    qid = row['id']
    if spec in topic_spec_dict:
        topic_spec_dict[spec].append(qid)
    else: 
        topic_spec_dict[spec] = [qid]

In [168]:
def kmeans(specialty):
    
    ids_list = topic_spec_dict[specialty]
    
    words = dict() # Map each word to its frequency in the corpus
    tokens = dict() # Map each word to its SpaCy token 
    for id in ids_list: #every id 
        for a in answers_dict[id] : #every answer under that question 
            doc = nlp(a) 
            # Count frequency of nouns in the corpus
            for token in doc:
                if token.is_alpha and token.pos_ == 'NOUN' :
                    if token.text not in words :
                        words[token.text] = 1
                        tokens[token.text] = token
                    else :
                        words[token.text] += 1

# Limit to nouns that occur at least 20 times
    freqwords = [w for w in words if words[w] > 10]   

#remove 1 character tokens/words in the dictionary 
    one_char_list_1 = []
    for i in words:
        if len(i) == 1:
            one_char_list_1.append(i)
    #print(i)
    for i in one_char_list_1:
        if i in words: del words[i]

    one_char_list_2 = []
    for i in tokens:
        if len(i) == 1:
            one_char_list_2.append(i)
    for i in one_char_list_2:
        if i in tokens: del tokens[i]
        
    for i in freqwords:
        if len(i) == 1: freqwords.remove(i)
            
    return freqwords, words, tokens


def showClust(k,clust) :
    for i in range(k) :
        vals = []
        for j in range(len(clust)) :
            if clust[j] == i :
                vals.append(freqwords[j])
        print(i,vals)   
        print(' ---------- ')

In [114]:
#Specialty name
df_docdx.name.unique()

array(['Dermatology', 'Gastroenterology', 'Pulmonology', 'Neurology',
       'Oncology / Hematology', 'Rheumatology', 'UNSPECIFIED / OTHER',
       'Anesthesia / Pain', 'Internal Medicine',
       'Endocrinology / Diabetes', 'Family Practice', 'Cardiology',
       'Ophthalmology / Optometrists'], dtype=object)

In [171]:
def pick_spe_kmeans(spec): 
    print(spec)
    print(' ')
    freqwords, words, tokens = kmeans('Oncology / Hematology')

    # Compare the frequent nouns to all the other nouns
    simwords = dict()
    for word in freqwords :
        # Sort the nouns by how similar they are
        sortedwords = sorted(words, key=lambda w: tokens[word].similarity(tokens[w]), reverse=True)
        # Take the top five most similar words
        simwords[word] = sortedwords[:5]
        

    # Show some examples
    for w in freqwords[:20] :
        print(w,simwords[w])
    
    # Get the vectors for the most frequent words
    vectors = []
    for word in freqwords :
        vectors.append(tokens[word].vector)
    print(' ----------------------------------------------------------- ')
    # Cluster the words into a fixed number of clusters
    K = 8 
    km = KMeans(n_clusters=K, init='k-means++', n_init=10,max_iter=1000)
    km.fit(vectors)
    x = km.fit_predict(vectors)

    # Show the words that belong to each cluster
    #showClust(K,x)
    return showClust(K,x)

### Final function 

In [172]:
pick_spe_kmeans('Oncology / Hematology')

Oncology / Hematology
 
xrt ['xrt', 'disorder', 'rituxn', 'chemoradiotherapy', 'line']
hospice ['hospice', 'abastrozole', 'refinement', 'safety', 'ice']
cns ['cns', 'jaundice', 'research', 'usa', 'soif']
histology ['histology', 'sedation', 'region', 'usage', 'histopathology']
tumor ['tumor', 'tumour', 'term', 'mire', 'metabolic']
splenectomy ['splenectomy', 'lumpectomymastectomy', 'spleenectomy', 'immunotherapy', 'hospicepalliative']
scan ['scan', 'ercp', 'rev', 'serologic', 'adenopathy']
revlimid ['revlimid', 'plasma', 'cranial', 'pulmonologist', 'flor']
profile ['profile', 'schedule', 'range', 'agnoist', 'context']
colonoscopy ['colonoscopy', 'bronchoscopy', 'context', 'prognosis', 'routinehow']
lesions ['lesions', 'levels', 'adjustments', 'declines', 'regulations']
metastasis ['metastasis', 'death', 'modality', 'osteoporosis', 'vector']
bone ['bone', 'alpha', 'adjuvent', 'tumour', 'ife']
use ['use', 'deformity', 'form', 'presence', 'sign']
cbc ['cbc', 'polyp', 'claims', 'metastases'

In [174]:
pick_spe_kmeans('Pulmonology')

Pulmonology
 
xrt ['xrt', 'disorder', 'rituxn', 'chemoradiotherapy', 'line']
hospice ['hospice', 'abastrozole', 'refinement', 'safety', 'ice']
cns ['cns', 'jaundice', 'research', 'usa', 'soif']
histology ['histology', 'sedation', 'region', 'usage', 'histopathology']
tumor ['tumor', 'tumour', 'term', 'mire', 'metabolic']
splenectomy ['splenectomy', 'lumpectomymastectomy', 'spleenectomy', 'immunotherapy', 'hospicepalliative']
scan ['scan', 'ercp', 'rev', 'serologic', 'adenopathy']
revlimid ['revlimid', 'plasma', 'cranial', 'pulmonologist', 'flor']
profile ['profile', 'schedule', 'range', 'agnoist', 'context']
colonoscopy ['colonoscopy', 'bronchoscopy', 'context', 'prognosis', 'routinehow']
lesions ['lesions', 'levels', 'adjustments', 'declines', 'regulations']
metastasis ['metastasis', 'death', 'modality', 'osteoporosis', 'vector']
bone ['bone', 'alpha', 'adjuvent', 'tumour', 'ife']
use ['use', 'deformity', 'form', 'presence', 'sign']
cbc ['cbc', 'polyp', 'claims', 'metastases', 'cts']
r

## 3. Topic Modeling - LDA
https://zhuanlan.zhihu.com/p/31470216


## Agenda

Ideal Answers: Producing relevant, precise, non-repetitive and readable
summaries for biomedical questions

1. Spacy with medical text 
2. Topic Modeling in LDA（algo method)


__Background:__

A topic model is a kind of a probabilistic generative model that has been used widely in the field of computer science with a specific focus on text mining and information retrieval in recent years. Since this model was first proposed, it has received a lot of attention and gained widespread interest among researchers in many research fields. 


__Topic modeling__ is a useful method (in contrast to the traditional means of data reduction in bioinformatics) and enhances researchers’ ability to interpret biological information.

The majority of medical documents and electronic health records (EHRs) are in text format that poses a challenge for data processing and finding relevant documents. Looking for ways to automatically retrieve the enormous amount of health and medical knowledge has always been an intriguing topic. Powerful methods have been developed in recent years to make the text processing automatic. One of the popular approaches to retrieve information based on discovering the themes in health & medical corpora is topic modeling; however, this approach still needs new perspectives.
- do research with remedy parterner

https://arxiv.org/pdf/1705.00995.pdf






Topic modeling is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using Latent Dirichlet Allocation or LDA, a popular approach to topic modeling.
In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a vector of token counts. 

There are two layers in this model — documents and tokens — and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)

They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.

The dimensions are fully indepedent from each other — there's no sense of connection between related tokens, such as knife and fork.
LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of topics, and the topics are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow Dirichlet probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

In [195]:
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features = n_features,
                                stop_words = 'english',
                                max_df = 0.5,
                                min_df = 10)


tf = tf_vectorizer.fit_transform(df_docdx_clean.text_np)                        

In [196]:
#modeling
n_topics = 20
lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 50,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)

In [197]:
#fit the model 
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [198]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print('')
    print()

In [199]:
n_top_words = 40
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
pain therapy physical consider pt management nsaids injection look joint help center injections topical muscle patch gabapentin patient cause benefit tramadol low andor gene program counseling specialist fracture tylenol massage ortho point acupuncture tens exercise imaging need helpful major chronic

Topic #1:
check cbc consider referral labs need echo thyroid including exam work ultrasound cmp symptoms esr evaluation blood rule start history renal panel crp workup ro negative ekg tsh eval rate lab disease physical pelvic ana function evaluate dvt studies stress

Topic #2:
trial months response monitor type suggest tx regimen clinical consider replacement poor recurrent inhibitor cymbalta suspicious recommend 46 based hedgehog trigger cycles localized complete beneficial strongly suspect markers active vismodegib lesion concurrent year underlying current infection scale controlled total frequent

Topic #3:
risk screening patients care age high years patient screen discuss 50

Plot and graph

In [200]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/Users/lexiewen1/anaconda/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     9.696445  1        1       0.111411 -0.125446
1      8.926539  1        2      -0.244226 -0.028886
5      8.762110  1        3       0.113369  0.213309
6      8.051331  1        4      -0.028283  0.133183
10     7.000710  1        5       0.209417  0.101586
0      6.103028  1        6      -0.000275  0.123234
3      5.796889  1        7       0.063066  0.238839
14     5.290255  1        8       0.064665 -0.100776
7      4.232534  1        9      -0.116789 -0.125107
16     4.176673  1        10      0.246757 -0.118718
13     4.098959  1        11     -0.083698 -0.138840
18     3.919735  1        12     -0.146768  0.166712
4      3.693030  1        13     -0.077943  0.097349
11     3.629330  1        14     -0.239777 -0.029594
12     3.442261  1        15      0.016545 -0.069641
8      3.211345  1        16     -0.089282 -0.053335
19     2.879322  1        17     -0.026391 -0.047948
9      2.755058  1        18      0.057731 -0.122062
2      2.232631  1        19      0.049427 -0.009720
15     2.101814  1        20      0.121044 -0.104141, topic_info=     Category         Freq              Term        Total  loglift  logprob
term                                                                       
115   Default  1040.000000  biopsy            1040.000000  30.0000  30.0000
600   Default  719.000000   needs             719.000000   29.0000  29.0000
588   Default  546.000000   mri               546.000000   28.0000  28.0000
643   Default  616.000000   pain              616.000000   27.0000  27.0000
471   Default  584.000000   insulin           584.000000   26.0000  26.0000
909   Default  990.000000   therapy           990.000000   25.0000  25.0000
654   Default  1123.000000  patient           1123.000000  24.0000  24.0000
934   Default  383.000000   trial             383.000000   23.0000  23.0000
685   Default  460.000000   possible          460.000000   22.0000  22.0000
904   Default  381.000000   testing           381.000000   21.0000  21.0000
923   Default  642.000000   topical           642.000000   20.0000  20.0000
284   Default  562.000000   dose              562.000000   19.0000  19.0000
959   Default  648.000000   use               648.000000   18.0000  18.0000
982   Default  531.000000   weeks             531.000000   17.0000  17.0000
763   Default  307.000000   refer             307.000000   16.0000  16.0000
414   Default  578.000000   history           578.000000   15.0000  15.0000
598   Default  942.000000   need              942.000000   14.0000  14.0000
191   Default  1666.000000  consider          1666.000000  13.0000  13.0000
928   Default  418.000000   treat             418.000000   12.0000  12.0000
797   Default  389.000000   risk              389.000000   11.0000  11.0000
521   Default  577.000000   like              577.000000   10.0000  10.0000
726   Default  404.000000   pt                404.000000   9.0000   9.0000 
941   Default  473.000000   try               473.000000   8.0000   8.0000 
861   Default  541.000000   steroid           541.000000   7.0000   7.0000 
110   Default  266.000000   better            266.000000   6.0000   6.0000 
672   Default  424.000000   physical          424.000000   5.0000   5.0000 
854   Default  870.000000   start             870.000000   4.0000   4.0000 
175   Default  300.000000   colonoscopy       300.000000   3.0000   3.0000 
577   Default  284.000000   mg                284.000000   2.0000   2.0000 
984   Default  432.000000   work              432.000000   1.0000   1.0000 
...       ...         ...   ..                       ...      ...      ... 
449   Topic20  209.713032   increase          210.665807   3.8578  -2.2984 
182   Topic20  63.242030    compression       64.194805    3.8474  -3.4971 
395   Topic20  43.931464    hair              44.884239    3.8409  -3.8615 
594   Topic20  41.477598    narcotics         42.430389

#### By Oncologist 

- 'Oncology / Hematology'

In [242]:
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features = n_features,
                                stop_words = 'english',
                                max_df = 0.5,
                                min_df = 10)


tf = tf_vectorizer.fit_transform(df_docdx_clean[df_docdx_clean['spe'] == 'Oncology / Hematology' ].text_np)          

#modeling
n_topics = 5
lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 50,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)

#fit the model 
lda.fit(tf)

n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
biopsy lung bone marrow colonoscopy ct pet colon bx scan need cancer petct cell nodules nodule transplant ultrasound recurrence negative

Topic #1:
consider therapy use treatment response patient inhibitor azedra alectinib radiation option options recommend brain splenectomy rituxan maintenance line parp control

Topic #2:
surgery staging ercp neoadjuvant biopsy chemo eus followed mri stent petct consult surgical disease resection chemotherapy evaluation gi ct diagnosis

Topic #3:
needs need center exam refer history check referral patient pain cmp cbc thalassemia pt physical dose major like iron beta

Topic #4:
chemotherapy adjuvant status chemo therapy need followed cycles radiation consider patient disease based recommend neoadjuvant cancer stage surgery xrt risk




In [243]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/Users/lexiewen1/anaconda/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      30.476299  1        1       0.014459  0.159425
1      21.205127  1        2       0.290862  0.064887
0      20.114014  1        3      -0.163502 -0.083462
2      15.818356  1        4      -0.201443  0.101226
3      12.386205  1        5       0.059623 -0.242076, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
50    Default  379.000000  biopsy        379.000000  30.0000  30.0000
268   Default  167.000000  lung          167.000000  29.0000  29.0000
54    Default  148.000000  bone          148.000000  28.0000  28.0000
280   Default  127.000000  marrow        127.000000  27.0000  27.0000
461   Default  175.000000  surgery       175.000000  26.0000  26.0000
448   Default  153.000000  status        153.000000  25.0000  25.0000
99    Default  103.000000  colonoscopy   103.000000  24.0000  24.0000
164   Default  88.000000   ercp          88.000000   23.0000  23.0000
307   Default  150.000000  neoadjuvant   150.000000  22.0000  22.0000
444   Default  124.000000  staging       124.000000  21.0000  21.0000
17    Default  167.000000  adjuvant      167.000000  20.0000  20.0000
84    Default  234.000000  chemo         234.000000  19.0000  19.0000
502   Default  88.000000   use           88.000000   18.0000  18.0000
170   Default  67.000000   eus           67.000000   17.0000  17.0000
304   Default  161.000000  needs         161.000000  16.0000  16.0000
450   Default  64.000000   stent         64.000000   15.0000  15.0000
87    Default  212.000000  chemotherapy  212.000000  14.0000  14.0000
302   Default  279.000000  need          279.000000  13.0000  13.0000
124   Default  99.000000   cycles        99.000000   12.0000  12.0000
98    Default  71.000000   colon         71.000000   11.0000  11.0000
78    Default  58.000000   center        58.000000   10.0000  10.0000
42    Default  68.000000   azedra        68.000000   9.0000   9.0000 
24    Default  68.000000   alectinib     68.000000   8.0000   8.0000 
347   Default  102.000000  petct         102.000000  7.0000   7.0000 
176   Default  48.000000   exam          48.000000   6.0000   6.0000 
190   Default  204.000000  followed      204.000000  5.0000   5.0000 
294   Default  105.000000  mri           105.000000  4.0000   4.0000 
406   Default  100.000000  response      100.000000  3.0000   3.0000 
63    Default  77.000000   bx            77.000000   2.0000   2.0000 
213   Default  46.000000   history       46.000000   1.0000   1.0000 
...       ...        ...   ..                  ...      ...      ... 
194   Topic5   12.817612   gene          13.606079   2.0289  -5.2234 
83    Topic5   12.477295   chelation     13.257860   2.0279  -5.2503 
430   Topic5   12.454825   short         13.237387   2.0276  -5.2521 
423   Topic5   12.083676   screen        12.864157   2.0260  -5.2823 
202   Topic5   10.799457   hb            11.583149   2.0185  -5.3947 
159   Topic5   10.426947   ekg           11.208727   2.0163  -5.4298 
31    Topic5   10.004611   annual        10.792215   2.0128  -5.4711 
254   Topic5   14.749644   levels        16.071748   2.0027  -5.0830 
78    Topic5   51.815234   center        58.248485   1.9716  -3.8265 
272   Topic5   27.163549   major         32.084269   1.9221  -4.4723 
47    Topic5   26.330245   beta          32.029957   1.8926  -4.5035 
394   Topic5   37.458120   referral      50.639532   1.7871  -4.1510 
5     Topic5   19.219025   50            24.801985   1.8336  -4.8183 
393   Topic5   45.963000   refer         78.948633   1.5476  -3.9463 
304   Topic5   72.293010   needs         161.569076  1.2844  -3.4935 
74    Topic5   29.579594   cbc           49.147049   1.5809  -4.3871 
12    Topic5   20.302259   add           28.090393   1.7639  -4.7635 
454   Topic5   20.983933   steroids      31.078303   1.6958  -4.7304 
148   Top

### By onocology questions 

In [244]:
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features = n_features,
                                stop_words = 'english',
                                max_df = 0.5,
                                min_df = 10)


tf = tf_vectorizer.fit_transform(df_docdx_clean[df_docdx_clean['name'] == 'Oncology / Hematology' ].text_np)          

#modeling
n_topics = 5
lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 50,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)

#fit the model 
lda.fit(tf)

n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
chemo status disease adjuvant need followed surgery chemotherapy therapy radiation consider neoadjuvant stage metastatic cycles staging mri patient months pet

Topic #1:
chemotherapy followed chemo recommend alectinib brain cancer based neoadjuvant mets cell gastric adjuvant stem resection start myeloma radiation surgery regimen

Topic #2:
consider therapy patient use inhibitor treatment response azedra option radiation options parp maintenance recommend clinical splenectomy rituxan patients line vismodegib

Topic #3:
biopsy lung marrow bone ct colonoscopy pet colon bx scan cancer need petct mammogram ultrasound nodules repeat diagnostic needs cbc

Topic #4:
ercp consult stent need refer center biopsy eus referral diagnosis oncology gi evaluation staging surgery resectable pancreatic tissue surgeon possible




In [245]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/Users/lexiewen1/anaconda/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      30.640873  1        1       0.114112 -0.089824
2      23.200745  1        2       0.134217  0.274237
3      20.230084  1        3      -0.165615 -0.071209
4      13.035408  1        4      -0.273706  0.036001
1      12.892889  1        5       0.190991 -0.149205, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
48    Default  392.000000  biopsy        392.000000  30.0000  30.0000
255   Default  166.000000  lung          166.000000  29.0000  29.0000
80    Default  244.000000  chemo         244.000000  28.0000  28.0000
182   Default  213.000000  followed      213.000000  27.0000  27.0000
83    Default  226.000000  chemotherapy  226.000000  26.0000  26.0000
158   Default  95.000000   ercp          95.000000   25.0000  25.0000
109   Default  83.000000   consult       83.000000   24.0000  24.0000
95    Default  103.000000  colonoscopy   103.000000  23.0000  23.0000
268   Default  134.000000  marrow        134.000000  22.0000  22.0000
119   Default  142.000000  ct            142.000000  21.0000  21.0000
425   Default  157.000000  status        157.000000  20.0000  20.0000
23    Default  70.000000   alectinib     70.000000   19.0000  19.0000
15    Default  169.000000  adjuvant      169.000000  18.0000  18.0000
427   Default  69.000000   stent         69.000000   17.0000  17.0000
293   Default  147.000000  neoadjuvant   147.000000  16.0000  16.0000
57    Default  108.000000  brain         108.000000  15.0000  15.0000
332   Default  158.000000  pet           158.000000  14.0000  14.0000
105   Default  280.000000  consider      280.000000  13.0000  13.0000
64    Default  149.000000  cancer        149.000000  12.0000  12.0000
76    Default  62.000000   center        62.000000   11.0000  11.0000
376   Default  78.000000   refer         78.000000   10.0000  10.0000
476   Default  86.000000   use           86.000000   9.0000   9.0000 
94    Default  75.000000   colon         75.000000   8.0000   8.0000 
436   Default  189.000000  surgery       189.000000  7.0000   7.0000 
164   Default  67.000000   eus           67.000000   6.0000   6.0000 
53    Default  157.000000  bone          157.000000  5.0000   5.0000 
377   Default  54.000000   referral      54.000000   4.0000   4.0000 
135   Default  83.000000   diagnosis     83.000000   3.0000   3.0000 
40    Default  107.000000  based         107.000000  2.0000   2.0000 
371   Default  201.000000  recommend     201.000000  1.0000   1.0000 
...       ...         ...    ...                ...     ...      ... 
29    Topic5   12.023997   appears       14.808976   1.8402  -5.3178 
272   Topic5   9.589084    metastases    11.925566   1.8304  -5.5441 
12    Topic5   9.235116    adeno         11.941140   1.7915  -5.5817 
498   Topic5   8.906888    zykadia       12.106083   1.7416  -5.6179 
418   Topic5   9.820001    sounds        13.779258   1.7098  -5.5203 
373   Topic5   14.804435   recommended   21.511786   1.6748  -5.1098 
349   Topic5   12.630245   preferred     18.848036   1.6482  -5.2686 
177   Topic5   20.287342   flot          30.841309   1.6296  -4.7947 
57    Topic5   62.351513   brain         108.512583  1.4944  -3.6719 
275   Topic5   48.128726   mets          82.889782   1.5049  -3.9308 
379   Topic5   26.768613   regimen       43.639971   1.5598  -4.5175 
83    Topic5   97.807930   chemotherapy  226.139450  1.2103  -3.2217 
182   Topic5   91.945173   followed      213.650305  1.2053  -3.2835 
40    Topic5   53.807432   based         107.724456  1.3543  -3.8193 
80    Topic5   82.845422   chemo         244.677476  0.9655  -3.3877 
64    Topic5   58.954654   cancer        149.061545  1.1209  -3.7279 
371   Topic5   71.197383   recommend     201.332821  1.0090  -3.5392 
293   Topic5   48.883296   neoadjuvant   147.830510  0.9419  -3.9153 
388   Top

## 4. Feature Selection
- can calculate the highest frequency words used (quantitative) and place into categories
    - phsu: drug name
    - dsyn: Disease or Syndrome
    - topp: Therapeutic or Preventive Procedure - treatment
    - fndg: finding 
    - diap: Diagnostic Procedure
    
    
- in two ways:
    1. overall level: by each question id
    2. individual level: by each answer/doctor

#### Load pre-built dictiionary 

In [218]:
import pickle
#load pre-built dictionary 
with open("dict_all_sep_10.txt", "rb") as myFile:
    dict_all = pickle.load(myFile)

### 1. By each question

In [180]:
#built in function - by question_id
def freq_umls(question_id, sem_type, Count):
    
    df = df_docdx_clean[df_docdx_clean['id'] == question_id]
    len_id = len(df)
    freq_dict = {}
    #count total frequency of answers - mentioned times 

        
    for index, row in df.iterrows():
        #print(index)
        for i in set(row.text_np.split()):
            if i in dict_all.keys() and dict_all[i][0] == sem_type:
                if dict_all[i][1] not in freq_dict:
                    freq_dict[dict_all[i][1]] = 1
                else:
                    freq_dict[dict_all[i][1]] += 1
                        
    if Count == True:  
        return sorted(freq_dict.items(), key=lambda x: x[1], reverse = True)
    
    else: 
        #freq_dict_per = {k: "{:.2%}".format(v / len_id) for k, v in freq_dict.items()}
        freq_dict_per = {k: (v / len_id) for k, v in freq_dict.items()}
        return sorted(freq_dict_per.items(), key=lambda x: x[1], reverse = True) 

#### Example

In [183]:
#shoe in count
freq_umls(88, 'phsu', True)

[('Insulin', 199),
 ('Lantus', 78),
 ('Sugars', 44),
 ('NovoLog', 44),
 ('Glucose', 28),
 ('Levemir', 28),
 ('Metformin', 16),
 ('Insulin Glargine', 8),
 ('Pharmaceutical Preparations', 7),
 ('Januvia', 5),
 ('Victoza', 4),
 ('Humalog', 3),
 ('Hypoglycemic Agents', 3),
 ('Agonist', 3),
 ('Sulfonylurea Compounds', 2),
 ('Insulins', 2),
 ('Linagliptin', 2),
 ('other medicated shampoos in ATC', 2),
 ('Inulin', 2),
 ('Histidine', 1),
 ('Incretins', 1),
 ('pioglitazone', 1),
 ('Water', 1),
 ('insulin degludec', 1),
 ('Jardiance', 1),
 ('Trulicity', 1),
 ('Hydroxymethylglutaryl-CoA Reductase Inhibitors', 1),
 ('Acarbose', 1),
 ('Fatty acid glycerol esters', 1),
 ('Bydureon', 1),
 ('Farxiga', 1)]

In [184]:
#show in percentage
freq_umls(88, 'phsu', False)

[('Insulin', 0.796),
 ('Lantus', 0.312),
 ('Sugars', 0.176),
 ('NovoLog', 0.176),
 ('Glucose', 0.112),
 ('Levemir', 0.112),
 ('Metformin', 0.064),
 ('Insulin Glargine', 0.032),
 ('Pharmaceutical Preparations', 0.028),
 ('Januvia', 0.02),
 ('Victoza', 0.016),
 ('Humalog', 0.012),
 ('Hypoglycemic Agents', 0.012),
 ('Agonist', 0.012),
 ('Sulfonylurea Compounds', 0.008),
 ('other medicated shampoos in ATC', 0.008),
 ('Linagliptin', 0.008),
 ('Inulin', 0.008),
 ('Insulins', 0.008),
 ('Histidine', 0.004),
 ('Farxiga', 0.004),
 ('pioglitazone', 0.004),
 ('Water', 0.004),
 ('Jardiance', 0.004),
 ('Trulicity', 0.004),
 ('Hydroxymethylglutaryl-CoA Reductase Inhibitors', 0.004),
 ('Incretins', 0.004),
 ('Acarbose', 0.004),
 ('Fatty acid glycerol esters', 0.004),
 ('Bydureon', 0.004),
 ('insulin degludec', 0.004)]

### 2. by each user_id 
- key word extraction

In [185]:
def feature_extraction(string, semtype):
    key_words = []
    for i in string.split():
        if i in dict_all.keys() and dict_all[i][0] == semtype:
            key_words.append(dict_all[i][1])
    return set(key_words)

In [186]:
def question_extraction(question_id, semtype):
    df = df_docdx[df_docdx['id'] == question_id]
    df = df.reset_index(drop=True)
    df['extraction'] = df['text_np'].apply(lambda x: feature_extraction(x, semtype))
    return df 

#### example

In [205]:
question_extraction(88, 'phsu').head(10)

,npi,first_name,last_name,email,created_at,id,title,description,text,name,text_np,extraction
0,1.760401e+09,Robert,Vyge,rvyge@msn.com,2018-07-23 10:40:54.092000+00:00,88,78-year-old female newly taking insulin reporting high levels of blood glucose in AM,"78-year-old female recently started on insulin, in addition to her 1000 mg BID Metformin. She was started on 15 units Novolog insulin prior to meals due to her review of blood sugars at 300s prior to mealtimes. Patient notes now her am blood glucose is high 100s, usually 180 and her before meals blood sugars are low 200s. She has also started walking about 20 minutes per day, after lunch. What do you recommend?","I would check her HbA1C I would add a Basal insulin at nighttime such as Levemir or Toujeo starting at 10u sc It would be important to know her weight and if she was obese , I would consider adding a GLP-1 in the am instead of the nighttime basal insulin",Internal Medicine,i would check her hba1c i would add a basal insulin at nighttime such as levemir or toujeo starting at 10u sc it would be important to know her weight and if she was obese i would consider adding a glp1 in the am instead of the nighttime basal insulin,"{Levemir, Insulin}"
1,1.780640e+09,Neal,Lakritz,nlakritz@massmed.org,2018-07-23 10:41:40.146000+00:00,88,78-year-old female newly taking insulin reporting high levels of blood glucose in AM,"78-year-old female recently started on insulin, in addition to her 1000 mg BID Metformin. She was started on 15 units Novolog insulin prior to meals due to her review of blood sugars at 300s prior to mealtimes. Patient notes now her am blood glucose is high 100s, usually 180 and her before meals blood sugars are low 200s. She has also started walking about 20 minutes per day, after lunch. What do you recommend?","there are many options here. The easiest from the patient standpoint would be to switch to a mixed insulin. That way she would get a longer duration of coverage. For example, NovoLog 70/30 , with the dose titrated upwards. this would require no additional injections. Other options would be to increase her current novolog, or add a GLP1",Internal Medicine,there are many options here the easiest from the patient standpoint would be to switch to a mixed insulin that way she would get a longer duration of coverage for example novolog 7030 with the dose titrated upwards this would require no additional injections other options would be to increase her current novolog or add a glp1,"{NovoLog, Insulin}"
2,1.386776e+09,Michael,Shanik,mshanikmd@hotmail.com,2018-07-23 10:46:55.837000+00:00,88,78-year-old female newly taking insulin reporting high levels of blood glucose in AM,"78-year-old female recently started on insulin, in addition to her 1000 mg BID Metformin. She was started on 15 units Novolog insulin prior to meals due to her review of blood sugars at 300s prior to mealtimes. Patient notes now her am blood glucose is high 100s, usually 180 and her before meals blood sugars are low 200s. She has also started walking about 20 minutes per day, after lunch. What do you recommend?","Several options, likely needs basal insulin as well.",Internal Medicine,several options likely needs basal insulin as well,{Insulin}
3,1.376568e+09,Randy,Shemer,shemer.r@comcast.net,2018-07-23 10:47:10.335000+00:00,88,78-year-old female newly taking insulin reporting high levels of blood glucose in AM,"78-year-old female recently started on insulin, in addition to her 1000 mg BID Metformin. She was started on 15 units Novolog insulin prior to meals due to her review of blood sugars at 300s prior to mealtimes. Patient notes now her am blood glucose is high 100s, usually 180 and her before meals blood sugars are low 200s. She has also started walking about 20 minutes per day, after lunch. What do you recommend?",Addition of a GLP-1 and possible an SGLT-2.,Internal Medicine,addition of a glp1 and possible an sglt2,{}
4,1.013947e+09,Gary,Vigilante,gary.vigilante@uphs.upenn.edu,2018-07-2

### 3. by each specialty of doctor

In [223]:
df_docdx_clean.columns

Index(['id', 'title', 'description', 'text_np', 'name', 'spe'], dtype='object')

In [224]:
#built in function - by question_id
def freq_umls(spe, sem_type, Count):
    
    df = df_docdx_clean[df_docdx_clean['spe'] == spe]
    len_id = len(df)
    freq_dict = {} 
    #count total frequency of answers - mentioned times 

        
    for index, row in df.iterrows():
        #print(index)
        for i in set(row.text_np.split()):
            if i in dict_all.keys() and dict_all[i][0] == sem_type:
                if dict_all[i][1] not in freq_dict:
                    freq_dict[dict_all[i][1]] = 1
                else:
                    freq_dict[dict_all[i][1]] += 1
                        
    if Count == True:  
        return sorted(freq_dict.items(), key=lambda x: x[1], reverse = True)
    
    else: 
        #freq_dict_per = {k: "{:.2%}".format(v / len_id) for k, v in freq_dict.items()}
        freq_dict_per = {k: (v / len_id) for k, v in freq_dict.items()}
        return sorted(freq_dict_per.items(), key=lambda x: x[1], reverse = True) 

In [227]:
freq_umls('Oncology / Hematology', 'phsu', True)

[('alectinib', 57),
 ('Rituxan', 46),
 ('Histidine', 38),
 ('Immunoglobulins, Intravenous', 35),
 ('rituximab', 33),
 ('Pharmaceutical Preparations', 32),
 ('vismodegib', 27),
 ('Tamoxifen', 25),
 ('Erivedge', 23),
 ('Promacta', 23),
 ('Carboplatin', 21),
 ('Cisplatin', 20),
 ('Iron', 20),
 ('Charcoal', 20),
 ('sonidegib', 19),
 ('fostamatinib', 19),
 ('Xgeva', 18),
 ('Taxol', 18),
 ('olaparib', 17),
 ('Fluorouracil', 17),
 ('ceritinib', 12),
 ('Alimta', 11),
 ('Oxygen', 11),
 ('Prednisone', 11),
 ('romiplostim', 10),
 ('Insulin', 10),
 ('Zometa', 9),
 ('Revlimid', 9),
 ('Thalidomide', 9),
 ('Lynparza', 9),
 ('other medicated shampoos in ATC', 9),
 ('eltrombopag', 8),
 ('agonists', 8),
 ('Herceptin', 8),
 ('Velcade', 8),
 ('Immunoglobulins', 8),
 ('Avid', 8),
 ('Anti-Inflammatory Agents, Non-Steroidal', 7),
 ('Diuretics', 7),
 ('Dexamethasone', 6),
 ('Prolia', 6),
 ('Etoposide', 6),
 ('Abraxane', 6),
 ('Agonist', 5),
 ('Alcohols', 5),
 ('Lithium', 5),
 ('Catecholamines', 5),
 ('Avastin

In [228]:
freq_umls('Oncology / Hematology', 'phsu', False)

[('alectinib', 0.031879194630872486),
 ('Rituxan', 0.025727069351230425),
 ('Histidine', 0.021252796420581657),
 ('Immunoglobulins, Intravenous', 0.019574944071588368),
 ('rituximab', 0.018456375838926176),
 ('Pharmaceutical Preparations', 0.017897091722595078),
 ('vismodegib', 0.015100671140939598),
 ('Tamoxifen', 0.013982102908277404),
 ('Erivedge', 0.012863534675615212),
 ('Promacta', 0.012863534675615212),
 ('Carboplatin', 0.01174496644295302),
 ('Cisplatin', 0.011185682326621925),
 ('Iron', 0.011185682326621925),
 ('Charcoal', 0.011185682326621925),
 ('sonidegib', 0.010626398210290829),
 ('fostamatinib', 0.010626398210290829),
 ('Xgeva', 0.010067114093959731),
 ('Taxol', 0.010067114093959731),
 ('olaparib', 0.009507829977628635),
 ('Fluorouracil', 0.009507829977628635),
 ('ceritinib', 0.006711409395973154),
 ('Alimta', 0.006152125279642058),
 ('Prednisone', 0.006152125279642058),
 ('Oxygen', 0.006152125279642058),
 ('romiplostim', 0.005592841163310962),
 ('Insulin', 0.005592841163

## 5. Question analysis 

Provide a detailed analysis of __Syntactic Structure__

The structure was used to determine whether each question is __“polar”__ (a yes/no question) or __“non-polar”__ (a question using “what”, “how” or “when”)

In addition, a summary of each question was obtained by __analyzing multiple clauses__ to determine, verbs, subjects and objects. 

These summaries are shown as __lists of words__ in square brackets. 



A set of __rules__ was written to assign each summary to one out of 5 categories: 
- drug (seeking names or classes of medications)
- treatment (more general issues about therapy)
- diagnosis (tests and workup of patients)
- communication (interaction with patients and providers)
- time (seeking durations, frequencies or other temporal information)




Each separate question below is classified as __polar (seeking positive or negative response)__ or __non-polar (seeking a specific type of answer)__

The questions are assigned to one of the 5 categories to assist in __identifying the type of
information to be expected in the response__ 



In [204]:
# Return questions that match given target
def getQuestionType(target,answertype) :
    q = dict()
    for qid in answertype :
        atype = answertype[qid]
        question = questions[qid]
        if target in atype :
            q[qid] = question
    return q

# Show tokens from parsing some text
def showTokens(qid) :
    question = questions[qid]
    doc = nlp(question)
    for token in doc:
        print(token.i,token.tag_,token.text,token.dep_,token.head.i)

# Get root notes from parsing some text
def getRoots(doc) :
    roots = []
    for token in doc:
        if token.dep_ == 'ROOT' :
            roots.append(token)
    return roots

# Determine if a root is a question
def isquest(root,doc) :
    for child in root.children :
        if child.text == '?' : return True
    return False

# Get main clause
def getClause(root,doc) :
    wh = getWh(root,doc)
    head = getHead(root,doc)
    args = getArgs(root,doc)
    prep = getPrep(root,doc)
    comps = getComps(root,doc)
    words = wh + head + args + prep + comps
    return words

# Get head of clause
def getHead(root,doc) :
    head = []
    h = root.lemma_
    if h not in ['be','find','have','suppose','do','need','about'] :
        head.append(h)
    return head

# Find wh-words (what, how, when)
def getWh(root,doc) :
    wh = []
    for child in root.children :
        if child.tag_ in ['WP', 'WDT', 'WRB'] :
            wh.append(child.lemma_)
        elif child.dep_ in ['advmod'] :
            for grandchild in child.children :
                if grandchild.tag_ in ['WP', 'WDT', 'WRB'] :
                    wh.extend([grandchild.lemma_,child.lemma_])
    return wh

# Get prepositional phrase
def getPrep(root,doc) :
    prep = []
    for child in root.children :
        if child.dep_ == 'prep' :
            prep.append(child.lemma_)
            for grandchild in child.children :
                if grandchild.dep_ == 'pobj' :
                    c = grandchild.lemma_
                    prep.append(c)
    return prep

# Get arguments of clause            
def getArgs(root,doc) :
    args = []
    for child in root.children :
        if child.dep_ in ['nsubj', 'dobj', 'iobj', 'nsubjpass', 'attr'] :
            #args.append((child.dep_,child.lemma_))
            c = child.lemma_
            if c not in ['-PRON-','anyone','someone','anything','that','what','who'] :
                wh = getWh(child,doc)
                mods = getMods(child,doc)
                prep = getPrep(child,doc)
                args = wh + [c] + mods + prep
    return args

# Get modifiers of noun
def getMods(root,doc) :
    mods = []
    for child in root.children :
        if child.dep_ in ['amod','compound'] :
            c = child.lemma_
            if c not in ['good'] :
                mods.append(c)
    return mods

# Get complements of a given root 
def getComps(root,doc) :
    comps = []
    for token in doc : 
        if token.dep_ in ['xcomp', 'acomp', 'relcl', 'advcl', 'pcomp', 'csubj'] :
            if root.is_ancestor(token) :
                wh = getWh(token,doc)
                head = getHead(token,doc)
                args = getArgs(token,doc)
                prep = getPrep(token,doc)
                words = wh + prep + head + args 
                comps.extend(words)
    return comps

# Map words to a category of question
def getCat(words) :
    polarity = 'non-polar'
    
    if 'what' in words :
        i = words.index('what') 
        if words[i+1] == 'treatment' :
            if words[i+2] in ['drug','pharmacological'] :
                cat = 'drug'
            else :
                cat = 'treatment'
        elif words[i+1] in ['agent','prescription','analgesic'] :
            cat = 'drug'
        elif words[i+1] in ['treat','precaution','thinking','regimen','patient'] :
            cat = 'treatment'
        elif words[i+1] in ['factor','sign','target'] :
            cat = 'diagnosis'
        elif words[i+1] in ['period'] :
            cat = 'time'
        else :
            cat = words[i+1]
            
    elif 'how' == words[0] :
        if  words[1] in ['often','time']:
            cat ='time'
        elif  words[1] in ['interpret']:
            cat = 'diagnosis'
        elif  words[1] in ['receive']:
            cat = 'communication'
        elif  words[1] in ['present']:
            cat = 'communication'
        elif  words[1] in ['decide']:
            cat = 'treatment'
        else :
            cat =  'how'
            
    elif 'when' == words[0] :
        cat =  'time'
        
    else :
        polarity = 'polar'
        if words[0] in ['agent'] :
            cat = 'drug'
        elif words[0] in ['restart','experience','history','success','add','way',
                'suggest','advocate','regimen','contraindication',
                'manage','role','monitoring','eradication','difference'] :
            cat = 'treatment'
        elif words[0] in ['discuss'] :
            cat = 'communication'
        elif words[0] in ['explain','biopsy','gender','recommend'] :
            cat = 'diagnosis'
        elif  words[0] in ['point']:
            cat ='time'
        else : 
            cat = 'treatment'
            
    return polarity, cat

# Get asnwer types, questions and answers    
answertype = getAnswerTypes()
questions, answers = getQandA(answertype)

# Determine categories for each question
for qid in questions :
    question = questions[qid]
    print(qid,question)
    doc = nlp(question) # parse the question
    roots = getRoots(doc) # get the root nodes
    for r in roots :
        if not isquest(r,doc) : continue # Determine if question
        words = getClause(r,doc) # Extract list of important words
        print(words)
        polar, cat = getCat(words) # Determine polarity and category
        print(polar,cat)
    print()

NameError: name 'getAnswerTypes' is not defined

### Apply to this into a pipeline 

In [ ]:
# This code provides a simple example of adding a UMLS pipeline to SpaCy.

import spacy
from spacy.tokens import Span
 
# Return UMLS semantic type for a given word 
# This function needs to be replaced by a real function that can map any term. 
def getUMLS(term) :
    #future dictionary 
    #umls = semtype_dict
    umls = {"The patient" : "podg", "treatment" : "topp", "a new drug" : "phsu" }
    if term in umls :
        return umls[term]
    else :
        return None
 
class UMLSPipeline(object):
    def __init__(self, nlp):
        Span.set_extension('umls', default=None)
 
    def __call__(self, doc):
        for chunk in doc.noun_chunks:
            sem = getUMLS(chunk.text)
            if sem != None:
                chunk._.set('umls', sem)
        return doc
 
 
nlp = spacy.load('en_core_web_md')  

# Create a UMLS pipeline and add to the existing ones
umls_pipeline = UMLSPipeline(nlp)

#customize the spacy nlp 
nlp.add_pipe(umls_pipeline, name='umls')
print(nlp.pipeline)

#doc = nlp("this is still in the acute phase and we cannot assume he will continue to have pain. the combination of 800 mg of ibuprofen and 650 mg of acetaminophen 3 times a day would be recommended. the use of oxycodone without acetaminophen 4 times a day whether the pain is elevated or not . the use of as needed increases the chances of chasing the pain which can increase the chances of addiction. if there is continued pain after surgeries after 6 months than a long-acting opioid may be indicated. ")
doc = nlp("The patient a new drug treatment with a new drug.")

#Show noun phrase chunks and their mapping to UMLS
for chunk in doc.noun_chunks:
    
    #remove a/the before the noun phrase
    #look up don't matched in every case 
    
        print(chunk.text,len(chunk.text),len(chunk),chunk._.umls) 

#chunk: how many words in the phrase
#chunk.text: how many charactors in the phrase 
 
#span : one or more tokens 
#multi-word tokens: span

#noun